<a href="https://colab.research.google.com/github/JohnPaul1278/therealps2/blob/main/ps2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import time, os, sys, re
import zipfile, json, datetime, string
import numpy as np
from statistics import *

import matplotlib.pyplot as plt

import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler


from google.colab import files

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter()

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

plt.style.use('classic')

In [13]:
dat=pd.read_csv('PRfoodsec.csv') #https://data.humdata.org/dataset/faostat-food-security-indicators-for-puerto-rico
#that's where I found this data.
#it's about food security on the island.
dat.head(5)
dat['Area Code'].to_string()
dat.dtypes

,Iso3,StartDate,EndDate,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag,Note
0,#country+code,#date+start,#date+end,NaN,NaN,#country+name,#indicator+code,#indicator+name,NaN,NaN,NaN,#date+year,#indicator+type,#indicator+value+num,NaN,NaN
1,PRI,2000-01-01,2000-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2000.0,2000,I$,30124.7,X,NaN
2,PRI,2001-01-01,2001-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2001.0,2001,I$,31958.1,X,NaN
3,PRI,2002-01-01,2002-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2002.0,2002,I$,32209.4,X,NaN
4,PRI,2003-01-01,2003-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2003.0,2003,I$,32206.5,X,NaN


'0        NaN\n1      177.0\n2      177.0\n3      177.0\n4      177.0\n5      177.0\n6      177.0\n7      177.0\n8      177.0\n9      177.0\n10     177.0\n11     177.0\n12     177.0\n13     177.0\n14     177.0\n15     177.0\n16     177.0\n17     177.0\n18     177.0\n19     177.0\n20     177.0\n21     177.0\n22     177.0\n23     177.0\n24     177.0\n25     177.0\n26     177.0\n27     177.0\n28     177.0\n29     177.0\n30     177.0\n31     177.0\n32     177.0\n33     177.0\n34     177.0\n35     177.0\n36     177.0\n37     177.0\n38     177.0\n39     177.0\n40     177.0\n41     177.0\n42     177.0\n43     177.0\n44     177.0\n45     177.0\n46     177.0\n47     177.0\n48     177.0\n49     177.0\n50     177.0\n51     177.0\n52     177.0\n53     177.0\n54     177.0\n55     177.0\n56     177.0\n57     177.0\n58     177.0\n59     177.0\n60     177.0\n61     177.0\n62     177.0\n63     177.0\n64     177.0\n65     177.0\n66     177.0\n67     177.0\n68     177.0\n69     177.0\n70     177.0\n71   

Iso3                object
StartDate           object
EndDate             object
Area Code          float64
Area Code (M49)     object
Area                object
Item Code           object
Item                object
Element Code       float64
Element             object
Year Code          float64
Year                object
Unit                object
Value               object
Flag                object
Note               float64
dtype: object

In [16]:
areacodesm49={'\'630': 'Cupey', '\'631': 'San Juan', '\'632': 'Bayamon'}
pandemic = {'2021': 'post', '2022': 'post', '2020': 'post', '2019': 'pre', '2018': 'pre', '2017': 'pre', '2016': 'pre', '2015': 'pre', '2014': 'pre', '2013': 'pre', '2012': 'pre', '2011': 'pre', '2010': 'pre', '2009': 'pre', '2008': 'pre', '2007': 'pre', '2006': 'pre', '2005': 'pre', '2004': 'pre', '2003': 'pre', '2002': 'pre', '2001': 'pre', '2000': 'pre'}
dat['Area Code (M49) name'] = dat['Area Code (M49)'].map(areacodesm49)
dat['Pandemic Era'] = dat['Year'].map(pandemic)
dat.head(5)
#mapped area code and mapped year. Not all the area codes show up but it was good practice to map

,Iso3,StartDate,EndDate,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag,Note,Area Code (M49) name,Pandemic Era
0,#country+code,#date+start,#date+end,NaN,NaN,#country+name,#indicator+code,#indicator+name,NaN,NaN,NaN,#date+year,#indicator+type,#indicator+value+num,NaN,NaN,NaN,NaN
1,PRI,2000-01-01,2000-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2000.0,2000,I$,30124.7,X,NaN,Cupey,pre
2,PRI,2001-01-01,2001-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2001.0,2001,I$,31958.1,X,NaN,Cupey,pre
3,PRI,2002-01-01,2002-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2002.0,2002,I$,32209.4,X,NaN,Cupey,pre
4,PRI,2003-01-01,2003-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2003.0,2003,I$,32206.5,X,NaN,Cupey,pre


In [18]:
dat.replace(to_replace='PRI', value='Puerto Rico')
dat.replace(to_replace='I$', value='$')
#both of these are fairly simple. PRI isn't intuitive to me so I changed it to the name Puerto Rico
#I also changed I$ to simply $ because 'I' was redundant given its use here

,Iso3,StartDate,EndDate,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag,Note,Area Code (M49) name,Pandemic Era
0,#country+code,#date+start,#date+end,NaN,NaN,#country+name,#indicator+code,#indicator+name,NaN,NaN,NaN,#date+year,#indicator+type,#indicator+value+num,NaN,NaN,NaN,NaN
1,Puerto Rico,2000-01-01,2000-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2000.0,2000,I$,30124.7,X,NaN,Cupey,pre
2,Puerto Rico,2001-01-01,2001-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2001.0,2001,I$,31958.1,X,NaN,Cupey,pre
3,Puerto Rico,2002-01-01,2002-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2002.0,2002,I$,32209.4,X,NaN,Cupey,pre
4,Puerto Rico,2003-01-01,2003-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2003.0,2003,I$,32206.5,X,NaN,Cupey,pre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,Puerto Rico,2018-01-01,2018-12-31,177.0,'630,Puerto Rico,21059,Incidence of caloric losses at retail distribu...,6121.0,Value,2018.0,2018,%,3.02,E,NaN,Cupey,pre
590,Puerto Rico,2019-01-01,2019-12-31,177.0,'630,Puerto Rico,21059,Incidence of caloric losses at retail distribu...,6121.0,Value,2019.0,2019,%,3.02,E,NaN,Cupey,pre
591,Puerto Rico,2020-01-01,2020-12-31,177.0,'630,Puerto Rico,21059,Incidence of caloric losses at retail distribu...,6121.0,Value,2020.0,2020,%,3.02,E,NaN,Cupey,post
592,Puerto Rico,2021-01-01,2021-12-31,177.0,'630,Puerto Rico,21059,Incidence of caloric losses at retail distribu...,6121.0,Value,2021.0,2021,%,3.02,E,NaN,Cupey,post


,Iso3,StartDate,EndDate,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag,Note,Area Code (M49) name,Pandemic Era
0,#country+code,#date+start,#date+end,NaN,NaN,#country+name,#indicator+code,#indicator+name,NaN,NaN,NaN,#date+year,#indicator+type,#indicator+value+num,NaN,NaN,NaN,NaN
1,PRI,2000-01-01,2000-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2000.0,2000,$,30124.7,X,NaN,Cupey,pre
2,PRI,2001-01-01,2001-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2001.0,2001,$,31958.1,X,NaN,Cupey,pre
3,PRI,2002-01-01,2002-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2002.0,2002,$,32209.4,X,NaN,Cupey,pre
4,PRI,2003-01-01,2003-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2003.0,2003,$,32206.5,X,NaN,Cupey,pre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,PRI,2018-01-01,2018-12-31,177.0,'630,Puerto Rico,21059,Incidence of caloric losses at retail distribu...,6121.0,Value,2018.0,2018,%,3.02,E,NaN,Cupey,pre
590,PRI,2019-01-01,2019-12-31,177.0,'630,Puerto Rico,21059,Incidence of caloric losses at retail distribu...,6121.0,Value,2019.0,2019,%,3.02,E,NaN,Cupey,pre
591,PRI,2020-01-01,2020-12-31,177.0,'630,Puerto Rico,21059,Incidence of caloric losses at retail distribu...,6121.0,Value,2020.0,2020,%,3.02,E,NaN,Cupey,post
592,PRI,2021-01-01,2021-12-31,177.0,'630,Puerto Rico,21059,Incidence of caloric losses at retail distribu...,6121.0,Value,2021.0,2021,%,3.02,E,NaN,Cupey,post


In [22]:
Dollar = dat.iloc[1:22]
Dollar.head(5)
Percent = dat.iloc[23:43]
Percent.head(5)
#we sliced here just to show different types of data. Some rows showed dollar values but some showed it in percentages.

,Iso3,StartDate,EndDate,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag,Note,Area Code (M49) name,Pandemic Era
1,PRI,2000-01-01,2000-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2000.0,2000,I$,30124.7,X,NaN,Cupey,pre
2,PRI,2001-01-01,2001-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2001.0,2001,I$,31958.1,X,NaN,Cupey,pre
3,PRI,2002-01-01,2002-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2002.0,2002,I$,32209.4,X,NaN,Cupey,pre
4,PRI,2003-01-01,2003-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2003.0,2003,I$,32206.5,X,NaN,Cupey,pre
5,PRI,2004-01-01,2004-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2004.0,2004,I$,35016.9,X,NaN,Cupey,pre


,Iso3,StartDate,EndDate,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag,Note,Area Code (M49) name,Pandemic Era
23,PRI,2000-01-01,2002-12-31,177.0,'630,Puerto Rico,210041,Prevalence of undernourishment (percent) (3-ye...,6121.0,Value,20002002.0,2002,%,NaN,E,NaN,Cupey,pre
24,PRI,2001-01-01,2003-12-31,177.0,'630,Puerto Rico,210041,Prevalence of undernourishment (percent) (3-ye...,6121.0,Value,20012003.0,2003,%,NaN,E,NaN,Cupey,pre
25,PRI,2002-01-01,2004-12-31,177.0,'630,Puerto Rico,210041,Prevalence of undernourishment (percent) (3-ye...,6121.0,Value,20022004.0,2004,%,NaN,E,NaN,Cupey,pre
26,PRI,2003-01-01,2005-12-31,177.0,'630,Puerto Rico,210041,Prevalence of undernourishment (percent) (3-ye...,6121.0,Value,20032005.0,2005,%,NaN,E,NaN,Cupey,pre
27,PRI,2004-01-01,2006-12-31,177.0,'630,Puerto Rico,210041,Prevalence of undernourishment (percent) (3-ye...,6121.0,Value,20042006.0,2006,%,NaN,E,NaN,Cupey,pre


In [26]:
GBUnit = dat.groupby('Unit')
GBUnit.head(5)

,Iso3,StartDate,EndDate,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag,Note,Area Code (M49) name,Pandemic Era
0,#country+code,#date+start,#date+end,NaN,NaN,#country+name,#indicator+code,#indicator+name,NaN,NaN,NaN,#date+year,#indicator+type,#indicator+value+num,NaN,NaN,NaN,NaN
1,PRI,2000-01-01,2000-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2000.0,2000,I$,30124.7,X,NaN,Cupey,pre
2,PRI,2001-01-01,2001-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2001.0,2001,I$,31958.1,X,NaN,Cupey,pre
3,PRI,2002-01-01,2002-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2002.0,2002,I$,32209.4,X,NaN,Cupey,pre
4,PRI,2003-01-01,2003-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2003.0,2003,I$,32206.5,X,NaN,Cupey,pre
5,PRI,2004-01-01,2004-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2004.0,2004,I$,35016.9,X,NaN,Cupey,pre
23,PRI,2000-01-01,2002-12-31,177.0,'630,Puerto Rico,210041,Prevalence of undernourishment (percent) (3-ye...,6121.0,Value,20002002.0,2002,%,NaN,E,NaN,Cupey,pre
24,PRI,2001-01-01,2003-12-31,177.0,'630,Puerto Rico,210041,Prevalence of undernourishment (percent) (3-ye...,6121.0,Value,20012003.0,2003,%,NaN,E,NaN,Cupey,pre
25,PRI,2002-01-01,2004-12-31,177.0,'630,Puerto Rico,210041,Prevalence of undernourishment (percent) (3-ye...,6121.0,Value,20022004.0,2004,%,NaN,E,NaN,Cupey,pre
26,PRI,2003-01-01,2005-12-31,177.0,'630,Puerto Rico,210041,Prevalence of undernourishment (percent) (3-ye...,6121.0,Value,20032005.0,2005,%,NaN,E,NaN,Cupey,pre


In [29]:
gbyear = dat.groupby('Item Code')
gbyear.head(5)

,Iso3,StartDate,EndDate,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag,Note,Area Code (M49) name,Pandemic Era
0,#country+code,#date+start,#date+end,NaN,NaN,#country+name,#indicator+code,#indicator+name,NaN,NaN,NaN,#date+year,#indicator+type,#indicator+value+num,NaN,NaN,NaN,NaN
1,PRI,2000-01-01,2000-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2000.0,2000,I$,30124.7,X,NaN,Cupey,pre
2,PRI,2001-01-01,2001-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2001.0,2001,I$,31958.1,X,NaN,Cupey,pre
3,PRI,2002-01-01,2002-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2002.0,2002,I$,32209.4,X,NaN,Cupey,pre
4,PRI,2003-01-01,2003-12-31,177.0,'630,Puerto Rico,22013,"Gross domestic product per capita, PPP, (const...",6126.0,Value,2003.0,2003,I$,32206.5,X,NaN,Cupey,pre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,PRI,2000-01-01,2000-12-31,177.0,'630,Puerto Rico,21059,Incidence of caloric losses at retail distribu...,6121.0,Value,2000.0,2000,%,3.01,E,NaN,Cupey,pre
572,PRI,2001-01-01,2001-12-31,177.0,'630,Puerto Rico,21059,Incidence of caloric losses at retail distribu...,6121.0,Value,2001.0,2001,%,3,E,NaN,Cupey,pre
573,PRI,2002-01-01,2002-12-31,177.0,'630,Puerto Rico,21059,Incidence of caloric losses at retail distribu...,6121.0,Value,2002.0,2002,%,3,E,NaN,Cupey,pre
574,PRI,2003-01-01,2003-12-31,177.0,'630,Puerto Rico,21059,Incidence of caloric losses at retail distribu...,6121.0,Value,2003.0,2003,%,3,E,NaN,Cupey,pre


In [30]:
dat=pd.read_csv('science-and-technology_pri.csv') #https://www.kaggle.com/datasets/fireballbyedimyrnmom/puerto-rico-dataset/
#this dataset had a couple files within it. I chose the QuickFacts file for this comparison but I may use others later.
#other files included crime rate information, climate change data and technological data
dat = dat.rename(columns={'Country Name':'Area'})

In [32]:
#df1=pd.read_csv('PRfoodsec.csv')
df2=pd.read_csv('science-and-technology_pri.csv')

In [33]:
pd.merge(dat,df2,how='outer')

,Area,Country ISO3,Year,Indicator Name,Indicator Code,Value,Country Name
0,#country+name,#country+code,#date+year,#indicator+name,#indicator+code,#indicator+value+num,#country+name
1,Puerto Rico,PRI,2015,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,0.42979,Puerto Rico
2,Puerto Rico,PRI,2013,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,0.43859,Puerto Rico
3,Puerto Rico,PRI,2009,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,0.45131,Puerto Rico
4,Puerto Rico,PRI,2016,Scientific and technical journal articles,IP.JRN.ARTC.SC,566,Puerto Rico
5,Puerto Rico,PRI,2015,Scientific and technical journal articles,IP.JRN.ARTC.SC,585.8,Puerto Rico
6,Puerto Rico,PRI,2014,Scientific and technical journal articles,IP.JRN.ARTC.SC,621.3,Puerto Rico
7,Puerto Rico,PRI,2013,Scientific and technical journal articles,IP.JRN.ARTC.SC,577.6,Puerto Rico
8,Puerto Rico,PRI,2012,Scientific and technical journal articles,IP.JRN.ARTC.SC,624.3,Puerto Rico
9,Puerto Rico,PRI,2011,Scientific and technical journal articles,IP.JRN.ARTC.SC,631.2,Puerto Rico


# New Section

In [10]:
merged_df.describe

<bound method NDFrame.describe of               Iso3    StartDate     EndDate  Area Code Area Code (M49)  \
0    #country+code  #date+start   #date+end        NaN             NaN   
1              PRI   2000-01-01  2000-12-31      177.0            '630   
2              PRI   2001-01-01  2001-12-31      177.0            '630   
3              PRI   2002-01-01  2002-12-31      177.0            '630   
4              PRI   2003-01-01  2003-12-31      177.0            '630   
..             ...          ...         ...        ...             ...   
611            NaN          NaN         NaN        NaN             NaN   
612            NaN          NaN         NaN        NaN             NaN   
613            NaN          NaN         NaN        NaN             NaN   
614            NaN          NaN         NaN        NaN             NaN   
615            NaN          NaN         NaN        NaN             NaN   

              Area        Item Code  \
0    #country+name  #indicator+code   